# Encontrando insights de la UEFA Champions League

La Liga de Campeones de la UEFA, a menudo llamada Liga de Campeones, es una competencia anual de fútbol por excelencia que cautiva a los fanáticos de todo el mundo. Establecida en 1955 como la Copa de Clubes Campeones de Europa, evolucionó a la Liga de Campeones de la UEFA en 1992, ampliando su atractivo. El formato moderno cuenta con 32 equipos de clubes de primer nivel seleccionados en función de su desempeño en la liga nacional, lo que aumenta la intriga.  

Este evento electrizante trasciende los deportes y se convierte en una celebración de la unidad, la cultura y el orgullo nacional. Los fanáticos, vestidos con los colores de sus países, crean una atmósfera eléctrica, lo que hace que el torneo sea tanto sobre los espectadores como sobre los jugadores. Financieramente, la Liga de Campeones es un salvavidas para los clubes, ya que aumenta los ingresos y ofrece oportunidades transformadoras. Sin embargo, genera debates sobre las disparidades de riqueza en el fútbol europeo.  

La Liga de Campeones es sinónimo de rivalidades históricas, triunfos de los desvalidos y brillantez individual. Para los jugadores, representa la cima de su carrera, mientras que para los fanáticos, es un fenómeno cultural. El himno y los rituales icónicos enriquecen la experiencia futbolística. En 200 palabras, la UEFA Champions League es el epítome de la excelencia del fútbol europeo, que ofrece momentos inolvidables, recompensas económicas y un impacto cultural único, con 32 clubes de primer nivel que se suman a su atractivo.

**Nombres de las tablas o ficheros a utilizar**:  
[uefa_2020.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EQuW8d16tZJKpYZN6jR480sBgc1IYypAw9hGHKQARS560g?e=7TybBU)  
[uefa_2021.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ESG8YBrjD3tBky3aGT0MwIABNt_PQxFVVYVlWMnZYUmV8g?e=VrYHtX)  
[uefa_2022.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/Ecf8O8U8sxJLoW9Bkd2ZVzgBb_gZc35mNcprnR9FMlzliQ?e=Vn3a6v)  
- Todas las tablas tienen las mismas columnas y data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` |

### Requerimientos:  
-Puedes utilizar SQL o PySpark

In [0]:
from pyspark.sql import SparkSession

# Iniciar una sesión de Spark
spark = SparkSession.builder.appName("UEFA_Analysis").getOrCreate()

1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: TEAM_NAME_HOME y TEAM_HOME_SCORE ordenadas en orden descendente de TEAM_HOME_SCORE. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

Primero, cargar el archivo CSV de la temporada 2020-2021 en un DataFrame (DF):

In [0]:
# Cargar el archivo en un DF
df_2020 = spark.read.option("header", "true") \
                    .option("inferSchema", "true") \
                    .csv("dbfs:/FileStore/PySpark-Data-Cleansing/Caso_2/uefa_2020.csv")

# Mostrar el esquema y el contenido del DF
df_2020.printSchema()
df_2020.display()

Utilizar las funciones `groupBy`, `agg` y `sum` para agrupar las filas según el nombre del equipo local, y añadir otra columna que sea la suma de `TEAM_HOME_SCORE` para ese nombre de equipo, ess decir, el total de goles que ha anotado el equipo jugando como local.

Después, utilizar `orderBy` con las opciones `desc` y `limit(3)` para ordenar los resultados según los 3 valores más altos de `TEAM_HOME_SCORE`.

In [0]:
from pyspark.sql.functions import sum, col

# Agrupar por TEAM_NAME_HOME y sumar los goles de cada equipo en casa
df_2020 = df_2020.groupBy("TEAM_NAME_HOME").agg(sum("TEAM_HOME_SCORE").alias("TEAM_HOME_SCORE"))

# Ordenar por el total de goles en casa en orden descendente y seleccionar los primeros 3
df_2020 = df_2020.orderBy(col("TEAM_HOME_SCORE").desc()).limit(3)

df_2020.display()

2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: TEAM_NAME y GAME_COUNT, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

Primero, cargar el archivo CSV de la temporada 2021-2022 en un DF:

In [0]:
# Cargar el archivo en un DF
df_2021 = spark.read.option("header", "true") \
                    .option("inferSchema", "true") \
                    .csv("dbfs:/FileStore/PySpark-Data-Cleansing/Caso_2/uefa_2021.csv")

# Mostrar el esquema y el contenido del DF
df_2021.printSchema()
df_2021.display()

Se utilizan las funciones `when` y `otherwise` para crear dos nuevas columnas, `MAJORITY_POSSESSION_HOME` y `MAJORITY_POSSESSION_AWAY`, que son `1` si el equipo tuvo posesión mayoritaria y `0` si no.

In [0]:
# Comparar las posesiones y filtrar los partidos con posesión mayoritaria
df_2021 = df_2021.withColumn(
    "MAJORITY_POSSESSION_HOME", 
    when(col("POSSESSION_HOME") > col("POSSESSION_AWAY"), 1).otherwise(0)
).withColumn(
    "MAJORITY_POSSESSION_AWAY", 
    when(col("POSSESSION_AWAY") > col("POSSESSION_HOME"), 1).otherwise(0)
)

df_2021.printSchema()
df_2021.display()

Usar `filter` para seleccionar solo aquellos partidos donde uno de los equipos tuvo posesión mayoritaria.

In [0]:
print(f"Registros antes de aplicar el filtro: {df_2021.count()}")

# Filtrar solo los partidos donde un equipo tuvo posesión mayoritaria
df_2021 = df_2021.filter(
    (col("MAJORITY_POSSESSION_HOME") == 1) | (col("MAJORITY_POSSESSION_AWAY") == 1)
)

print(f"Registros después de aplicar el filtro: {df_2021.count()}")

Seleccionar el nombre del equipo que tuvo posesión mayoritaria en cada partido usando `when`, y luego contar la cantidad de veces que aparece cada equipo.

In [0]:
# Contar cuántas veces tuvo posesión mayoritaria cada equipo
df_2021 = df_2021.select(
    when(col("MAJORITY_POSSESSION_HOME") == 1, col("TEAM_NAME_HOME"))
    .when(col("MAJORITY_POSSESSION_AWAY") == 1, col("TEAM_NAME_AWAY"))
    .alias("TEAM_NAME")
).groupBy("TEAM_NAME").count().withColumnRenamed("count", "GAME_COUNT")

# Ordenar por la cantidad de partidos (GAME_COUNT) en orden descendente y tomar el primer equipo
df_2021 = df_2021.orderBy(col("GAME_COUNT").desc()).limit(1)

df_2021.display()

3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.

Primero, cargar el archivo CSV de la temporada 2022-2023 en un DF:

In [0]:
# Cargar el archivo en un DF
df_2022 = spark.read.option("header", "true") \
                    .option("inferSchema", "true") \
                    .csv("dbfs:/FileStore/PySpark-Data-Cleansing/Caso_2/uefa_2022.csv")

# Mostrar el esquema y el contenido del DF
df_2022.printSchema()
df_2022.display()

Primero, se debe eliminar el signo de porcentaje (%) para los valores de `DUELS_WON_HOME` y `DUELS_WON_AWAY` y convertir los valores a números para poder hacer comparaciones.

In [0]:
from pyspark.sql.functions import regexp_replace

# Convertir los porcentajes en las columnas 'DUELS_WON_HOME' y 'DUELS_WON_AWAY' a valores numéricos
df_2022 = df_2022.withColumn(
    "DUELS_WON_HOME_PERCENT", 
    regexp_replace(col("DUELS_WON_HOME"), "%", "").cast("float") / 100
)
df_2022 = df_2022.withColumn(
    "DUELS_WON_AWAY_PERCENT", 
    regexp_replace(col("DUELS_WON_AWAY"), "%", "").cast("float") / 100
)

df_2022.printSchema()
df_2022.display()

Comparar los valores de los porcentajes de los duelos ganados de cada equipo y decidir si un equipo ganó el duelo.

In [0]:
# Condición para identificar el equipo que ganó el duelo pero perdió el partido
df_2022 = df_2022.withColumn(
    "TEAM_LOST", 
    when(
        (col("DUELS_WON_HOME_PERCENT") > col("DUELS_WON_AWAY_PERCENT")) & \
            (col("TEAM_HOME_SCORE") < col("TEAM_AWAY_SCORE")), col("TEAM_NAME_HOME")
    ).when(
        (col("DUELS_WON_AWAY_PERCENT") > col("DUELS_WON_HOME_PERCENT")) & \
            (col("TEAM_AWAY_SCORE") < col("TEAM_HOME_SCORE")), col("TEAM_NAME_AWAY")
    ).otherwise("(DRAW)")  # Mostrar que hubo empate
)

df_2022.printSchema()
df_2022.display()

Usamos `filter` para mantener solo las filas donde se cumple la condición de que un equipo ganó el duelo pero perdió el partido, y seleccionar las columnas `STAGE` y `TEAM_LOST`

In [0]:
# Filtrar solo los registros donde hay un equipo que ganó el duelo pero perdió el partido
df_2022 = df_2022.filter(col("TEAM_LOST") != "(DRAW)")

# Seleccionar las columnas relevantes y mostrar los resultados por fase (STAGE)
df_2022 = df_2022.select("STAGE", "TEAM_LOST")

df_2022.display()